# SageMaker

## fetch training data

```
my_training_classes
├── person
│   ├── han.jpg
│   ├── leia.jpg
|   ├── luke.jpg
│   └── . . .
└── ship
│   ├── millenium_falcon.jpg
│   ├── tie-fighter.jpg    
│   ├── x-wing.jpg
│   ├── . . .
└── . . .
```

In [11]:
# S3 Bucket Name
data_bucket_name='deeplens-sagemaker-kbc'

# prefix name inside the S3 bucket containing train_rec 
dataset_name = 'original' 

## Setting up the environment


In [12]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()
sess = sagemaker.Session()

training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")

## Preparing data for our model

### Find the im2rec.py script on this system


In [13]:
# Find im2rec in our environment and set up some other vars in our environemnt

base_dir='/tmp'

%env BASE_DIR=$base_dir
%env S3_DATA_BUCKET_NAME = $data_bucket_name
%env DATASET_NAME = $dataset_name

import sys,os

suffix='/mxnet/tools/im2rec.py'
im2rec = list(filter( (lambda x: os.path.isfile(x + suffix )), sys.path))[0] + suffix
%env IM2REC=$im2rec

env: BASE_DIR=/tmp
env: S3_DATA_BUCKET_NAME=deeplens-sagemaker-kbc
env: DATASET_NAME=original
env: IM2REC=/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/tools/im2rec.py


### Get our training images from S3
In order to create training and validation RecordIO files, we need to download our images to our local filesystem.

In [14]:
# Pull our images from S3
!aws s3 sync s3://$S3_DATA_BUCKET_NAME/public/$DATASET_NAME $BASE_DIR/$DATASET_NAME --quiet

### Create RecordIO files from training images


In [15]:
%%bash
# Use the IM2REC script to convert our images into RecordIO files

# Clean up our working dir of existing LST and REC files
cd $BASE_DIR
rm *.rec
rm *.lst

# First we need to create two LST files (training and test lists), noting the correct label class for each image
# We'll also save the output of the LST files command, since it includes a list of all of our label classes
echo "Creating LST files"
python $IM2REC --list --recursive --pass-through --test-ratio=0.3 --train-ratio=0.7 $DATASET_NAME $DATASET_NAME > ${DATASET_NAME}_classes

echo "Label classes:"
cat ${DATASET_NAME}_classes

# Then we create RecordIO files from the LST files
echo "Creating RecordIO files"
python $IM2REC --num-thread=4 ${DATASET_NAME}_train.lst $DATASET_NAME
python $IM2REC --num-thread=4 ${DATASET_NAME}_test.lst $DATASET_NAME
ls -lh *.rec

Creating LST files
Label classes:
bord 0
cup 1
deep 2
empty_bord 3
nespresso 4
nothing 5
oreo 6
shure 7
small_tr 8
yellow 9
Creating RecordIO files
Creating .rec file from /tmp/original_train.lst in /tmp
time: 0.033582210540771484  count: 0
time: 161.09551882743835  count: 1000
Creating .rec file from /tmp/original_test.lst in /tmp
time: 2.240488290786743  count: 0
-rw-rw-r-- 1 ec2-user ec2-user 472M Dec 11 12:42 original_test.rec
-rw-rw-r-- 1 ec2-user ec2-user 1.2G Dec 11 12:41 original_train.rec


### Upload our training and test data RecordIO files so we can train with them
Now that we have our training and test .rec files, we upload them to S3 so SageMaker can use them for training

In [16]:
# Upload our train and test RecordIO files to S3 in the bucket that our sagemaker session is using
bucket = 'deeplens-sagemaker-kbc'

s3train_path = 's3://{}/{}/train/'.format(bucket, dataset_name)
s3validation_path = 's3://{}/{}/validation/'.format(bucket, dataset_name)

# Clean up any existing data
!aws s3 rm s3://{bucket}/{dataset_name}/train --recursive
!aws s3 rm s3://{bucket}/{dataset_name}/validation --recursive

# Upload the rec files to the train and validation channels
!aws s3 cp /tmp/{dataset_name}_train.rec $s3train_path
!aws s3 cp /tmp/{dataset_name}_test.rec $s3validation_path

upload: ../../../tmp/original_train.rec to s3://deeplens-sagemaker-kbc/original/train/original_train.rec
upload: ../../../tmp/original_test.rec to s3://deeplens-sagemaker-kbc/original/validation/original_test.rec


### Configure the data for our model training to use
Finally, we tell SageMaker where to find these RecordIO files to use for training

In [17]:
train_data = sagemaker.session.s3_input(
    s3train_path, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

validation_data = sagemaker.session.s3_input(
    s3validation_path, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

data_channels = {'train': train_data, 'validation': validation_data}

## Training
Now it's time to train our model!

### Create an image classifier object with some base configuration
More info here: https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.Estimator

In [18]:
s3_output_location = 's3://{}/{}/output'.format(bucket, dataset_name)

image_classifier = sagemaker.estimator.Estimator(
    training_image,
    role, 
    train_instance_count=1, 
    train_instance_type='ml.p2.xlarge',
    output_path=s3_output_location,
    sagemaker_session=sess
)

### Set some training hyperparameters

Finally, before we train, we provide some additional configuration parameters for the training.

More info here: https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html

In [19]:
num_classes=! ls -l {base_dir}/{dataset_name} | wc -l
num_classes=int(num_classes[0]) - 1

num_training_samples=! cat {base_dir}/{dataset_name}_train.lst | wc -l
num_training_samples = int(num_training_samples[0])

# Learn more about the Sagemaker built-in Image Classifier hyperparameters here: https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html

# These hyperparameters we won't want to change, as they define things like
# the size of the images we'll be sending for input, the number of training classes we have, etc.
base_hyperparameters=dict(
    use_pretrained_model=1,
    image_shape='3,224,224',
    num_classes=num_classes,
    num_training_samples=num_training_samples,
    epochs = 2
)

# These are hyperparameters we may want to tune, as they can affect the model training success:
hyperparameters={
    **base_hyperparameters, 
    **dict(
        learning_rate=0.001,
        mini_batch_size=5,
    )
}


image_classifier.set_hyperparameters(**hyperparameters)

hyperparameters

{'use_pretrained_model': 1,
 'image_shape': '3,224,224',
 'num_classes': 10,
 'num_training_samples': 1061,
 'epochs': 2,
 'learning_rate': 0.001,
 'mini_batch_size': 5}

In [ ]:
hyperparameters = dict(
    augmentation_type="crop_color_transform",
    beta_1 = 0.9, #used for adam optimizer
    beta_2 = 0.999, #used for adam optimizer
    epochs = 5,
    image_shape = "3,224,224",
    learning_rate = 0.1,
    lr_scheduler_factor = 0.1,
    lr_scheduler_step = 3,
    mini_batch_size = 32,
    momentum = 0.9,
    num_classes = 5, #variable
    num_layers = 50, # The algorithm supports multiple network depth (number of layers). They are 18, 34, 50, 101, 152 and 200\num_training_samples	10000
    num_training_samples = 10000, #variable
    optimizer = "adam",
    precision_dtype = "float32",
    use_pretrained_model = 1,
    weight_decay = 0.0001
)

### Start the training
Train our model!

This will take some time because it's provisioning a new container runtime to train our model, then the actual training happens, then the trained model gets uploaded to S3 and the container is shut down.

More info here: https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.Estimator.fit

In [20]:
%%time

import time
now = str(int(time.time()))
training_job_name = 'IC-' + dataset_name.replace('_', '-') + '-' + now

image_classifier.fit(inputs=data_channels, job_name=training_job_name, logs=True)

job = image_classifier.latest_training_job
model_path = f"{base_dir}/{job.name}"

print(f"\n\n Finished training! The model is available for download at: {image_claussifier.output_path}/{job.name}/output/model.tar.gz")

2019-12-11 12:43:11 Starting - Starting the training job...
2019-12-11 12:43:13 Starting - Launching requested ML instances......
2019-12-11 12:44:20 Starting - Preparing the instances for training.........
2019-12-11 12:46:11 Downloading - Downloading input data.........
2019-12-11 12:47:34 Training - Downloading the training image..Docker entrypoint called with argument(s): train

2019-12-11 12:47:54 Training - Training image download completed. Training in progress.[12/11/2019 12:47:59 INFO 139625756972864] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/image_classification/default-input.json: {u'beta_1': 0.9, u'gamma': 0.9, u'beta_2': 0.999, u'optimizer': u'sgd', u'use_pretrained_model': 0, u'eps': 1e-08, u'epochs': 30, u'lr_scheduler_factor': 0.1, u'num_layers': 152, u'image_shape': u'3,224,224', u'precision_dtype': u'float32', u'mini_batch_size': 32, u'weight_decay': 0.0001, u'learning_rate': 0.1, u'momentum': 0}
[12/11/2019 12:47:59 INFO 1396257569728

In [ ]:
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, CategoricalParameter, ContinuousParameter
hyperparameter_ranges = {'optimizer': CategoricalParameter(['sgd', 'adam']),
                         'learning_rate': ContinuousParameter(0.0001, 0.1),
                         'mini_batch_size': IntegerParameter(2, 32),
                        }

objective_metric_name = 'validation:accuracy'

tuner = HyperparameterTuner(image_classifier,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=50,
                            max_parallel_jobs=3)

tuner.fit(inputs=data_channels, logs=True, include_cls_metadata=False)